In [1]:
import numpy as np
import tncontract as tn
import qutip as qt
import time

from itertools import product
from loguru import logger

from src.simulation_utils import *

In [2]:
n_sites = 4 # Number of sites (qubits)
cct_depth = 100 # Circuit depth
chi = int(2 * np.log2(n_sites)) # Limit for the bond dimension

psi = tn.onedim.init_mps_allzero(n_sites, 2) # Initialize all qubits in zero

In [3]:
t = time.time()

for j in range(cct_depth):
    one_qubit_layer = random_single_qubit_gate_layer(n_sites)
    two_qubit_layer = random_two_qubit_gate_ladder(n_sites)[np.mod(j, 2)]

    psi = tn.onedim.contract_mps_mpo(psi, one_qubit_layer)
    psi.left_canonise(chi=chi)

    psi = tn.onedim.contract_mps_mpo(psi, two_qubit_layer)
    psi.left_canonise(chi=chi)

psi.left_canonise(normalise=True)
logger.info("Took {:.2f}s to run {} circuits on {} qubits".format(time.time()-t, cct_depth, n_sites))

2021-07-10 17:29:08.574 | INFO     | __main__:<module>:14 - Took 0.45s to run 100 circuits on 4 qubits


In [4]:
for s in psi:
    logger.info(s.shape)

2021-07-10 17:29:08.581 | INFO     | __main__:<module>:2 - (2, 1, 2)
2021-07-10 17:29:08.582 | INFO     | __main__:<module>:2 - (2, 2, 4)
2021-07-10 17:29:08.583 | INFO     | __main__:<module>:2 - (2, 4, 4)
2021-07-10 17:29:08.584 | INFO     | __main__:<module>:2 - (4, 2, 1)


In [5]:
psi_vec = tn.onedim.contract_virtual_indices(psi)
psi_vec.fuse_indices('physout','physout')

In [6]:
psi_vec.shape

(16,)

In [7]:
qt.Qobj(psi_vec.data.reshape(-1,1))

Quantum object: dims = [[16], [1]], shape = (16, 1), type = ket
Qobj data =
[[ 0.14772059+0.06411574j]
 [-0.0428185 +0.1494485j ]
 [ 0.14526236+0.07270105j]
 [-0.51313758+0.07886241j]
 [-0.03683685-0.06786437j]
 [ 0.03792712+0.2566511j ]
 [-0.25142519-0.09347721j]
 [ 0.09960068-0.13599757j]
 [-0.12372587+0.0995797j ]
 [-0.12766597+0.02639792j]
 [-0.22513899-0.0272516j ]
 [ 0.20192505-0.45517227j]
 [-0.06251565+0.03464566j]
 [ 0.22935023-0.20936559j]
 [ 0.09561556+0.10861925j]
 [-0.11254927+0.05989353j]]